# Beer_type Prediction -  FastAPI and Heroku Deployment

The project is to deploy a Machine Learning model into production. I will train a custom neural networks model that will accurately predict a type of beer based on some rating criterias such as appearance, aroma, palate or taste. I will also build a web app and deploy it online in order to serve my model for real time predictions. This notebook is summarizing the steps to design the web app using FastAPI and steps to deploy on Heroku.

Detailed git steps are not listed, assuming readers have basic git knowledge. Noted that some model's state_dict exceeds the 100MB file size limit on Github, to store it on GitHub, refer to this website https://git-lfs.github.com./. Also need to follow this link https://github.com/raxod502/heroku-buildpack-git-lfs to enable heroku to successfully deploy any app using git lfs.

I have tried and succeeded compressing the web app's size below 500MB (Heroku's size limit), but when the app is running, it requires a 516MB memory which exceeds the Heroku limit again. Therefore the final model I used on deployment is the 7 layers model(6-2048-1512-1024-512-256-128-104): train 39.8 | val 47.7 | test 47.5.

**Student Name:** Wenying Wu 

**Student Number:** 14007025

**Prerequisite:**
- Saved Neural Network Model State_dict from Neural Network Model.ipynb, label encoder, robust scaler, brewery_name_dict and beer_style_dict from Data Preparation.ipynb
- Registered on Heroku
- Github account and knowledge 

**Sections:**
1. Preparation
2. FastAPI Design
3. Heroku Deployment steps

## 1. Preparation

### 1.1 Create a new folder 'api' in previously created cookie-cutter style folder

### 1.2 Copy Paste below files into api folder

- brewery_name_dict.json
- beer_style_dict.json
- numeric_scaler.joblib
- brewery_name_encoder.joblib
- nn_final_dict.pt

### 1.3 Create pytorchmulticlass.py and fill in below content (the model class)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PytorchMultiClass(nn.Module):
    def __init__(self, num_features):
        super(PytorchMultiClass, self).__init__()
        
        self.layer_1 = nn.Linear(num_features, 2048) 
        self.layer_2 = nn.Linear(2048, 1512)
        self.layer_3 = nn.Linear(1512, 1024)
        self.layer_4 = nn.Linear(1024, 512)
        self.layer_5 = nn.Linear(512, 256)
        self.layer_6 = nn.Linear(256, 128)
        self.layer_out = nn.Linear(128, 104)
        self.softmax = nn.Softmax(dim=1)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.batchnorm1 = nn.BatchNorm1d(2048)
        self.batchnorm2 = nn.BatchNorm1d(1512)
        self.batchnorm3 = nn.BatchNorm1d(1024)
        self.batchnorm4 = nn.BatchNorm1d(512)
        self.batchnorm5 = nn.BatchNorm1d(256)
        self.batchnorm6 = nn.BatchNorm1d(128)


    def forward(self, x):
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        
        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_4(x)
        x = self.batchnorm4(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_5(x)
        x = self.batchnorm5(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_6(x)
        x = self.batchnorm6(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_out(x)
        return x # nn.CrossEntropyLoss does log_softmax() for us so we can simply return x

### 1.4 Create requirement.txt and fill in below content

-f https://download.pytorch.org/whl/torch_stable.html

Cython==0.29.24

fastapi==0.75.0

gunicorn==20.1.0

joblib==1.1.0

numpy==1.21.2

pandas==1.3.4

starlette==0.17.1

torch==1.8.1+cpu 

torchvision==0.9.1+cpu

uvicorn==0.17.6

scikit-learn==1.0.1



**Note:** 
- This step is for building docker image and Heroku app. If you want to develop and test your API locally on your machine, you can skip this step and pip/pipenv install above packages
- The first line above in used speciffically for heroku. As Heroku has a limitation on app size (500MB), our web app will go over this size limit if we install the gpu version of pytorch. The first line tells heroku to find the torch wersion table and hence we can specify a particular version.

### 1.5 Copy paste main.py in this link: https://github.com/WenyingWu-1/ADSI_AST2

The codes in main.py are self explained. and detailed explaination can be found in part 2.

## 2. FastAPI Design

### 2.1 API Endpoints included

- '/'(GET): "Displaying a brief description of the project objectives, list of endpoints, expected input parameters and output format of the model, link to the Github repo related to this project", 
- '/health/' (GET): "Returning status code 200 with a string with a welcome message", 
- '/brewer/'(GET): "Returning all avaialable brewers", 
- '/brewer/valid'(POST): "Check for single brewer_name validity",
- '/beer/type/'(POST): "Returning prediction for a single input only", 
- '/beers/type/'(POST): "Returning predictions for multiple inputs", 
- '/model/architecture/' (GET): "Displaying the architecture of your Neural Networks (listing of all layers with their types)"
- '/docs/'(GET): "Go to the docs page and use the app!"

**Note:** Can always to to app_url/docs (https://adsi-ast2-14007025.herokuapp.com/docs/)to test app, it is more convienient than typing queries in browser. 

### 2.2 Some Function's logic

- Read in saved Neural Network Model State_dict from Neural Network Model.ipynb, label encoder, robust scaler, brewery_name_dict and beer_style_dict from Data Preparation.ipynb using read_files() function
- Define functions: format_features() and multi_format_features() to format inputs. format_features() are used for a single input, it first check the validity of brewery_name, return False is brewert_name is invalid and the API will return a string telling user the brewery_name is not valid. If the validity check is passed, this fuction will transform inputs into pandas dataframe then encode brewery_name and scale other numeric inputs using the loaded encoder and robusr scaler. The fuction lastly returns a list of formatted model-ready inputs to the API. format_features() uses the same logic, the only difference is that it takes all inputs as string and separate them into lists. It also performs another check on the length of each input field, if the lengths of each input field are not the same, API will return a string report to user.
- Other than those endpoints in Assignment brief, 2 more endpoints are added for ease of use: '/brewer/'(GET) and '/brewer/valid'(POST). '/brewer/'(GET) simply returns a dictionary to user indicating all valid inputs for brewery_name field. While '/brewer/valid'(POST) expects a single string input, corresponding string will be returned to user regarding whether the input is checked as valid brewery_name.

### 2.3 Endpoints with no expected input

- '/'(GET)
- '/health/' (GET)
- '/brewer/'(GET)
- '/model/architecture/' (GET)
- '/docs/'(GET)

### 2.4 Endpoints with expected input

#### 2.4.1 '/brewer/valid'(POST)

**Input:** brewery_name: str

**Output:** String

- If valid: 'Brewer is valid, go predict!'
- If invalid: 'Brewer not valid, please select another brewer...'

#### 2.4.2 '/beer/type/'(POST):

**Input:** brewery_name: str, review_aroma: float, review_appearance: float, review_palate: float, review_taste: float, beer_abv: float

**Output:** String

- If all inputs are valid: Name of beer type e.g. "(512) Brewing Company"
- If brewery_name is invalid: "Please check your brewery_name input, it is probably wrong. Refer '/brewer/'(GET) for available brewer names or check via '/brewer/valid'(POST)..."

#### 2.4.3 '/beers/type/'(POST):

**Input:** brewery_name: str, review_aroma: str, review_appearance: str, review_palate: str, review_taste: str, beer_abv: str

**Output:** Dictionary

- If all inputs are valid: Names of beer type e.g. {"beer_type for input 1": "Belgian Pale Ale", "beer_type for input 2": "English Brown Ale"}
- If brewery_name is invalid: "Please check your brewery_name input, it is probably wrong. Refer '/brewer/'(GET) for available brewer names or check via '/brewer/valid'(POST)..."
- If lengths of fields are not the same or separation is incorrect: "Please ensure all blanks are inputted with same number of inputs. And make sure each input is separated by a comma and a blank space : 'input_1, input_2'"

**NOTE:** Please use **a comma and a blank** to separate inputs.

Examples: 

**brewery_a, brewery_b, brewery_c** for brewery_name field


**score_1, score_2, score_3** for other fields

### 2.5 API Usage

**Usage_1 (Recommended):**
1. Go to app_url/docs (https://adsi-ast2-14007025.herokuapp.com/docs/) in browser
2. Click on the command you like to execute
3. Click on 'Try it out'
4. Scroll down and the output will be in 'Response body'

**Usage_2:**
1. Go to app_url/endpoint_name (https://adsi-ast2-14007025.herokuapp.com/endpoint_name) (inputs required for post commands)
2. Result will be shown in browser

## 3. Heroku Deployment

### 3.1 Create a Procfile in the 'api' folder with below content

web: gunicorn -w 4 -k uvicorn.workers.UvicornWorker main:app

**Note:** Without this file Heroku will not be able to deploy the app.

### 3.2 Register a free account on Heroku https://www.heroku.com

### 3.3 Click on 'Create New App' button

- CreateEnter App name as your choice (I created it as 'adsi-ast2-14007025')
- Select any available region (I selected United States)
- Click on 'Create App' button

### 3.4 Click on 'Github' in Deployment method section

- Click on 'connect to github'
- Select the repositry (in my case 'adsi_ast2_api')
- Click on connect
- Select the branch to be deployed (in my case is 'master')
- Click on 'Enable automatic deploy' (depends on your choice)
- Click on 'Deploy' button
- Wait until Heroku tells you the app is deployed
- Copy the url (in my case https://adsi-ast2-14007025.herokuapp.com/) shown on the screen and paste it in your browser, your app in running on Heroku platform!
**Note:** For conveinient tests/ better UI, go to https://adsi-ast2-14007025.herokuapp.com/docs/